### 背包问题

#### 1. 贪心算法

挑选价值密度高的商品是一个很直观的比较好的选择。

In [4]:
import pylab

class Item(object):
    def __init__(self, n, v, w):
        self.name = n
        self.value = float(v)
        self.weight = float(w)
    def getName(self):
        return self.name
    def getValue(self):
        return self.value
    def getWeight(self):
        return self.weight
    def __str__(self):
        result = '<' + self.name + ', ' + str(self.value) + ', '\
                 + str(self.weight) + '>'
        return result

#建立物品序列
def buildItems():
    names = ['clock', 'painting', 'radio', 'vase', 'book',
             'computer']
    vals = [175,90,20,50,10,200]
    weights = [10,9,4,2,1,20]
    Items = []
    for i in range(len(vals)):
        Items.append(Item(names[i], vals[i], weights[i]))
    return Items

#贪心算法主函数
def greedy(Items, maxWeight, keyFcn):
    assert type(Items) == list and maxWeight >= 0#测试，确保Items是list，最大可装载量非负
    ItemsCopy = sorted(Items, key=keyFcn, reverse = True)#该排序规则下，最经济的物品排在最前
    result = []
    totalVal = 0.0
    totalWeight = 0.0
    i = 0#基于i进行循环，尽可能多地装入经济的物品
    while totalWeight < maxWeight and i < len(Items):#加入物品i后，数量和重量均有空间
        if (totalWeight + ItemsCopy[i].getWeight()) <= maxWeight:
            result.append((ItemsCopy[i]))
            totalWeight += ItemsCopy[i].getWeight()
            totalVal += ItemsCopy[i].getValue()
        i += 1
    return (result, totalVal)

def value(item):
    return item.getValue()

def weightInverse(item):
    return 1.0/item.getWeight()

def density(item):#价值密度
    return item.getValue()/item.getWeight()

def testGreedy(Items, constraint, getKey):
    taken, val = greedy(Items, constraint, getKey)
    print ('Total value of items taken = ' + str(val))
    for item in taken:
        print '  ', item
        
def testGreedys(maxWeight = 20):
    Items = buildItems()
    print('Items to choose from:')
    for item in Items:
        print '  ', item
    print 'Use greedy by value to fill a knapsack of size', maxWeight
    testGreedy(Items, maxWeight, value)#选值钱的
    print 'Use greedy by weight to fill a knapsack of size', maxWeight
    testGreedy(Items, maxWeight, weightInverse)#选轻的
    print 'Use greedy by density to fill a knapsack of size', maxWeight
    testGreedy(Items, maxWeight, density)#选价值密度高的    

In [5]:
testGreedys()

Items to choose from:
   <clock, 175.0, 10.0>
   <painting, 90.0, 9.0>
   <radio, 20.0, 4.0>
   <vase, 50.0, 2.0>
   <book, 10.0, 1.0>
   <computer, 200.0, 20.0>
Use greedy by value to fill a knapsack of size 20
Total value of items taken = 200.0
   <computer, 200.0, 20.0>
Use greedy by weight to fill a knapsack of size 20
Total value of items taken = 170.0
   <book, 10.0, 1.0>
   <vase, 50.0, 2.0>
   <radio, 20.0, 4.0>
   <painting, 90.0, 9.0>
Use greedy by density to fill a knapsack of size 20
Total value of items taken = 255.0
   <vase, 50.0, 2.0>
   <clock, 175.0, 10.0>
   <book, 10.0, 1.0>
   <radio, 20.0, 4.0>


然而，同样很直观地，这一选择太过随意，实际上它很可能不是最优的。要挑选出最优的物品集，一种绝对无遗漏且保证正确的方法便是生成所有可能的物品集，并从中选出最好的。这依赖于一种可以高效生成和存储可能物品集的机制。通常，1和0可以表示两种状态。在此用1和0表示“选”和“没选”该物品，自然地，物品集能被表示为二进制数，即0和1的序列。

#### 2. 暴力方法

In [43]:
def dToB(n, numDigits):
    """requires: n is a natural number less than 2**numDigits
      returns a binary string of length numDigits representing the
              the decimal number n."""
    assert type(n)==int and type(numDigits)==int and n >=0 and n < 2**numDigits
    bStr = ''
    while n > 0:
        bStr = str(n % 2) + bStr
        n = n//2
    while numDigits - len(bStr) > 0:
        bStr = '0' + bStr
    return bStr

def genPset(Items):
    """Generate a list of lists representing the power set of Items"""
    numSubsets = 2**len(Items)
    templates = []
    for i in range(numSubsets):
        templates.append(dToB(i, len(Items)))
    pset = []
    for t in templates:
        elem = []
        for j in range(len(t)):
            if t[j] == '1':
                elem.append(Items[j])
        pset.append(elem)
    return pset

def chooseBest(pset, constraint, getVal, getWeight):
    bestVal = 0.0
    bestSet = None
    for Items in pset:
        ItemsVal = 0.0
        ItemsWeight = 0.0
        for item in Items:
            ItemsVal += getVal(item)
            ItemsWeight += getWeight(item)
        if ItemsWeight <= constraint and ItemsVal > bestVal:
            bestVal = ItemsVal
            bestSet = Items
    return (bestSet, bestVal)

def testBest():
    Items = buildItems()
    pset = genPset(Items)
    taken, val = chooseBest(pset, 20, Item.getValue, Item.getWeight)
    print ('Total value of items taken = ' + str(val))
    for item in taken:
        print '  ', item

逐个函数考察。显然`dToB`将整数转化为二进制表达。

In [20]:
dToB(128,10)

'0010000000'

`genPset`生成一个列表，列表中的每一个元素也是一个列表，该列表内装着几个物品（的地址）。显然，`genPset`和`dToB`共同使得“对物品的选择”得以用一个二进制序列的形式便捷地存储，其中`genPset`函数体的前半部分很便捷地生成了类似于物品“全排列”的东西。

In [26]:
genPset(buildItems())[7]

In [27]:
for item in genPset(buildItems())[7]:
    print item

<vase, 50.0, 2.0>
<book, 10.0, 1.0>
<computer, 200.0, 20.0>


`chooseBest`函数十分直观。遍历物品集列表，选出最好的。由`testBest`负责结果的输出。

In [28]:
testBest()

Total value of items taken = 275.0
   <clock, 175.0, 10.0>
   <painting, 90.0, 9.0>
   <book, 10.0, 1.0>


产生幂集是很费时的（2^n）。是否有办法不扫全表？可以用递归。

#### 3. 递归方法（决策树）

决策树的思路如下：对于某一物品，只有两种选择：带与不带。比较带上该物品时的最优情况与不带上该物品时的最优情况，能保证得出正确的解。

In [30]:
def maxVal(toConsider, avail): 
    if toConsider == [] or avail == 0:
        result = (0, ())
    elif toConsider[0].getWeight() > avail:
        result = maxVal(toConsider[1:], avail)
    else:
        nextItem = toConsider[0]

        #Explore left branch
        #带上该物品
        withVal, withToTake = maxVal(toConsider[1:],
                                      avail - nextItem.getWeight())
        withVal += nextItem.getValue()
        #Explore right branch
        #不带上该物品
        withoutVal, withoutToTake = maxVal(toConsider[1:], avail)

        #Choose better branch
        #比较两种选择
        if withVal > withoutVal:
            result = (withVal, withToTake + (nextItem,))
        else:
            result = (withoutVal, withoutToTake)
    return result

def smallTest():
    Items = buildItems()
    val, taken = maxVal(Items, 20)
    for item in taken:
        print(item)
    print ('Total value of items taken = ' + str(val))

In [31]:
smallTest()

<book, 10.0, 1.0>
<painting, 90.0, 9.0>
<clock, 175.0, 10.0>
Total value of items taken = 275.0


### L8 Problem 2

In [41]:
names = ['Dirt', 'Computer', 'Fork', 'Problem Set']
vals = [0, 30, 1, -10]
weights = [4, 10, 5, 0]
Items = []
for i in range(len(vals)):
    Items.append(Item(names[i], vals[i], weights[i]))
val, taken = maxVal(Items, 14)
for item in taken:
    print item

<Computer, 30.0, 10.0>


In [42]:
def weightInverse(item):
    return -item.getWeight()
maxWeight = 14
print 'Use greedy by value to fill a knapsack of size', maxWeight
testGreedy(Items, maxWeight, value)#选值钱的
print 'Use greedy by weight to fill a knapsack of size', maxWeight
testGreedy(Items, maxWeight, weightInverse)#选重的！
print 'Use greedy by density to fill a knapsack of size', maxWeight
testGreedy(Items, maxWeight, density)#选价值密度高的   

Use greedy by value to fill a knapsack of size 14
Total value of items taken = 30.0
   <Computer, 30.0, 10.0>
   <Dirt, 0.0, 4.0>
Use greedy by weight to fill a knapsack of size 14
Total value of items taken = -9.0
   <Problem Set, -10.0, 0.0>
   <Dirt, 0.0, 4.0>
   <Fork, 1.0, 5.0>
Use greedy by density to fill a knapsack of size 14


ZeroDivisionError: float division by zero

### L8 Problem 4


关于`yield`的用法，参见http://pyzh.readthedocs.org/en/latest/the-python-yield-keyword-explained.html

In [54]:
# generate all combinations of N items
def powerSet(items):
    N = len(items)
    # enumerate the 2**N possible combinations
    for i in xrange(2**N):
        combo = []
        for j in xrange(N):
            # test bit jth of integer i(检验第i位是否为1)
            #e.g., i = 3 = (11), j = 0 and 1 
            if (i >> j) % 2 == 1:
                combo.append(items[j])
        yield combo

for item in powerSet('ABC'):
    print item

[]
['A']
['B']
['A', 'B']
['C']
['A', 'C']
['B', 'C']
['A', 'B', 'C']


逐步分析：

- i = 0 => []
- i = 1, j = 0 => A
- i = 2, j = 1 => B
- i = 3, j = 0 & j = 1 => AB
- i = 4. j = 2 => C
- i = 5, j = 0 & j = 2 => AC
- i = 6, j = 1 & j = 2 => BC
- i = 7, j = 0 & j = 1 & j = 2 => ABC

实际上，i提供了所有可能的情况：[],0,1,10,11,...，而j分别将每种情况翻译成输出。例如，对于情况 i = 101，j(0,2)将其翻译为AC，而j(0,2)是由i唯一确定的(第0位和第2位是1)。由以上分析，容易得到下面的实现。

In [60]:
def yieldAllCombos(items):
    """
      Generates all combinations of N items into two bags, whereby each 
      item is in one or zero bags.

      Yields a tuple, (bag1, bag2), where each bag is represented as 
      a list of which item(s) are in each bag.
    """
    N = len(items)
    # enumerate the 3**N possible combinations
    for i in xrange(3**N):
        combo = ([], [])
        for j in xrange(N):
            # test bit jth of integer i
            if (i / (3**j)) % 3 == 1:
                combo[0].append(items[j])
            elif (i / (3**j)) % 3 == 2:
                combo[1].append(items[j])
        yield combo

for item in yieldAllCombos('ABC'):
    print item

([], [])
(['A'], [])
([], ['A'])
(['B'], [])
(['A', 'B'], [])
(['B'], ['A'])
([], ['B'])
(['A'], ['B'])
([], ['A', 'B'])
(['C'], [])
(['A', 'C'], [])
(['C'], ['A'])
(['B', 'C'], [])
(['A', 'B', 'C'], [])
(['B', 'C'], ['A'])
(['C'], ['B'])
(['A', 'C'], ['B'])
(['C'], ['A', 'B'])
([], ['C'])
(['A'], ['C'])
([], ['A', 'C'])
(['B'], ['C'])
(['A', 'B'], ['C'])
(['B'], ['A', 'C'])
([], ['B', 'C'])
(['A'], ['B', 'C'])
([], ['A', 'B', 'C'])


### L8 Problem 5

In [65]:
from itertools import chain, combinations

def powerset(iterable):
    "powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(len(s)+1))

for subset in powerset([1,2,3]):
    print subset

()
(1,)
(2,)
(3,)
(1, 2)
(1, 3)
(2, 3)
(1, 2, 3)


### 斐波那契数列

对于斐波那契数列的递归算法，出现大量的重复计算。可以用记忆的办法（字典）减少计算次数。容易发现，不加记忆的递归算法的费时是难以忍受的。

In [70]:
import time

In [79]:
def fastFib(x, memo):
    assert type(x) == int and x >= 0 and type(memo) == dict
    if x == 0 or x == 1:
        return 1
    if x in memo:
        return memo[x]
    result = fastFib(x-1, memo) + fastFib(x-2, memo)
    memo[x] = result
    return result

def testFastFib(n):
    assert type(n) == int and n >=  0
    start = time.time()
    for i in range(n):
        tmp = fastFib(i, {})
    end = time.time()
    print 'Time cost: ', end - start

In [80]:
testFastFib(100)

Time cost:  0.00699996948242


In [83]:
def fib(x):
    assert type(x) == int and x >= 0
    if x == 0 or x == 1:
        return 1
    else:
        return fib(x-1) + fib(x-2)

def testFib(n):
    assert type(n) == int and n >=  0
    start = time.time()
    for i in range(n):
        tmp = fib(i)
    end = time.time()
    print 'Time cost: ', end - start

In [85]:
testFib(40)

Time cost:  160.902999878


对比迭代的斐波那契数列生成方法。

In [101]:
def iter(n, k, a, b):
    assert type(n) == int and n >= 0 and k <= n
    if n == 0 or n == 1:
        return 1
    elif k == n:
        return a
    else:
        return iter(n, k + 1, b, a + b)

def iterFib(n):
    return iter(n, 0, 1, 1)
        
def testIterFib(n):
    assert type(n) == int and n >= 0
    start = time.time()
    for i in range(n):
        tmp = iterFib(i)
    end = time.time()
    print 'Time cost: ', end - start

In [103]:
testIterFib(100)

Time cost:  0.00300002098083


能发现，迭代方法比记忆法更快。